<a href="https://colab.research.google.com/github/ferbator/data-engineering/blob/main/M33071_%D0%9A%D0%B0%D1%80%D0%B5%D0%BF%D0%B8%D0%BD_%22lab5%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/ML/data/women-clothing-accessories.csv", sep="\t", usecols=[0, 1])

In [ ]:
data

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
data

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
data = data.applymap(lambda s:s.lower() if type(s) == str else s)

data['review'] = data.apply(lambda row: nltk.word_tokenize(row['review']), axis=1)

data['review'] = data['review'].apply(lambda row: [pymorphy2_analyzer.parse(item)[0].normal_form 
                                                   for item in row])

stop = stopwords.words('russian')
data['review'] = data['review'].apply(lambda row: [item for item in row if item not in stop])

data['review'] = [' '.join(map(str, l)) for l in data['review']]

data

,review,sentiment
0,качество плохой пошив ужасный ( горловина напе...,negative
1,"товар отдать человек , получить посылка . ладн...",negative
2,"ужасный синтетик ! тонкий , общий представить ...",negative
3,"товар прийти , продавец продлить защита соглас...",negative
4,"кофточка голый синтетик , носить возможно .",negative
...,...,...
89995,сделать достаточно . ткань сделать рисунок зам...,positive
89996,накидка шикарный . спасибо большой провдо линя...,positive
89997,спасибо большой ) продовца рекомендовать .. за...,positive
89998,очень довольный заказ ! маленький месяц рб . к...,positive


In [ ]:
# Сформируйте датафрейм, в котором в строках будут разные векторайзеры, 
# а в столбцах разные метрики качества, а в ячейках будут значения этих метрик 
# для соответсвующих векторайзеров.

metrics = pd.DataFrame({'Accuracy_score':[0, 0, 0], 'Precision_score':[0, 0, 0], 'f1_score':[0, 0, 0], 'recall_score':[0, 0, 0]}, index=['CountVectorizer', 'Tf-idf', 'Символьные n-граммы'])

metrics

,Accuracy_score,Precision_score,f1_score,recall_score
CountVectorizer,0,0,0,0
Tf-idf,0,0,0,0
Символьные n-граммы,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7, random_state=10)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer()), ('clf', MultinomialNB())])

params = {'vect__ngram_range': [(1, 1), (1, 2), (1, 5)]}

GSCV = GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=2)
GSCV.fit(x_train, y_train)

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__ngram_range': [(1, 1), (1, 2), (1, 5)]})

In [ ]:
# count_vect = CountVectorizer(ngram_range=(1, 4))
# x_train = count_vect.fit_transform(x_train)

# clf = MultinomialNB()
# clf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

In [ ]:
# x_test = count_vect.transform(x_test)

# pred = clf.predict(x_test)

pred = GSCV.predict(x_test)


In [ ]:
metrics.loc['CountVectorizer'] = [accuracy_score(y_test, pred), precision_score(y_test, pred, average='macro'), f1_score(y_test, pred, average='macro'), recall_score(y_test, pred, average='micro')]

In [ ]:
metrics

,Accuracy_score,Precision_score,f1_score,recall_score
CountVectorizer,0.717159,0.718982,0.716629,0.717159
Tf-idf,0.000000,0.000000,0.000000,0.000000
Символьные n-граммы,0.000000,0.000000,0.000000,0.000000


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7, random_state=10)

In [ ]:
pipeline = Pipeline([('vect', TfidfVectorizer()), ('clf', MultinomialNB())])

params = {
    'vect__max_df': [0.5, 1.0, 1.5],
    'vect__min_df': [0.5, 1],
    'vect__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4)],
    'vect__max_features': [None, 10, 15, 100]
}

GSCV = GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=2)
GSCV.fit(x_train, y_train)

In [ ]:
# tfidf_vect = TfidfVectorizer(ngram_range=(1, 3), max_df=1.0, min_df=1, max_features = None)
# x_train = tfidf_vect.fit_transform(x_train)

# clf.fit(x_train, y_train)

In [ ]:
# x_test = tfidf_vect.transform(x_test)

# pred1 = clf.predict(x_test)
pred1 = GSCV.predict(x_test)

In [ ]:
metrics.loc['Tf-idf'] = [accuracy_score(y_test, pred1), precision_score(y_test, pred1, average='macro'), f1_score(y_test, pred1, average='macro'), recall_score(y_test, pred1, average='micro')]

In [ ]:
metrics

,Accuracy_score,Precision_score,f1_score,recall_score
CountVectorizer,0.717159,0.718982,0.716629,0.717159
Tf-idf,0.717159,0.718982,0.716629,0.717159
Символьные n-граммы,0.000000,0.000000,0.000000,0.000000


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7, random_state=10)

In [ ]:
pipeline = Pipeline([('vect', CountVectorizer(analyzer='char')), ('clf', MultinomialNB())])

params = {'vect__ngram_range': [(1, 1), (4, 6), (5, 7), (4, 7)]}

GSCV = GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=2)
GSCV.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(analyzer='char')),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__ngram_range': [(1, 1), (4, 6), (5, 7), (4, 7)]})

In [ ]:
# char_vect = CountVectorizer(analyzer='char', ngram_range=(4, 7))
# x_train = char_vect.fit_transform(x_train)

# clf.fit(x_train, y_train)

In [ ]:
# x_test = char_vect.transform(x_test)

# pred2 = clf.predict(x_test)

pred2 = GSCV.predict(x_test)

In [ ]:
metrics.loc['Символьные n-граммы'] = [accuracy_score(y_test, pred2), precision_score(y_test, pred2, average='macro'), f1_score(y_test, pred2, average='macro'), recall_score(y_test, pred2, average='micro')]

In [ ]:
metrics

,Accuracy_score,Precision_score,f1_score,recall_score
CountVectorizer,0.717159,0.718982,0.716629,0.717159
Tf-idf,0.717159,0.718982,0.716629,0.717159
Символьные n-граммы,0.718788,0.726610,0.720127,0.718788


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'\b[A-Za-z]{2}', "My name is Yoshikage Kira. I am 33 years old. My house is in the northeast section of Morioh, where all the villas are, and I am not married. I work as an employee for the Kame Yu department stores, and I get home every day by 8 PM at the latest. I do not smoke, but I occasionally drink. I am in bed by 11 PM, and make sure I get eight hours of sleep, no matter what.") 
print(result)

['My', 'na', 'is', 'Yo', 'Ki', 'am', 'ye', 'ol', 'My', 'ho', 'is', 'in', 'th', 'no', 'se', 'of', 'Mo', 'wh', 'al', 'th', 'vi', 'ar', 'an', 'am', 'no', 'ma', 'wo', 'as', 'an', 'em', 'fo', 'th', 'Ka', 'Yu', 'de', 'st', 'an', 'ge', 'ho', 'ev', 'da', 'by', 'PM', 'at', 'th', 'la', 'do', 'no', 'sm', 'bu', 'oc', 'dr', 'am', 'in', 'be', 'by', 'PM', 'an', 'ma', 'su', 'ge', 'ei', 'ho', 'of', 'sl', 'no', 'ma', 'wh']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
string = "My name is Yoshikage Kira. I'm 33 years old. My house is in the northeast section of Morioh, where all the villas are, and I am not married. I work as an employee for the Kame Yu department stores, and I get home every day by 8 PM at the latest. I don't smoke, but I occasionally drink. I'm in bed by 11 PM, and make sure I get eight hours of sleep, no matter what."
re.split("\.", string, maxsplit=2)

['My name is Yoshikage Kira',
 " I'm 33 years old",
 " My house is in the northeast section of Morioh, where all the villas are, and I am not married. I work as an employee for the Kame Yu department stores, and I get home every day by 8 PM at the latest. I don't smoke, but I occasionally drink. I'm in bed by 11 PM, and make sure I get eight hours of sleep, no matter what."]

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
string = "126712523dsgyuafsdy"
re.sub(r"\d", "DIG", string)

'DIGDIGDIGDIGDIGDIGDIGDIGDIGdsgyuafsdy'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
string = "Collab: https://colab.research.google.com/drive/1XMppVajelSMBqSeJYminJ5mDsy_WOoRw#scrollTo=KwNS9zt4WhAv"
re.sub(r"https?:\/\/.*", '', string)

'Collab: '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
string = "chap01"
prog = re.compile(r'chap[124]')
prog.findall(string)

[]

In [ ]:
string = "My name is Yoshikage Kira. I'm 33 years old. My house is in the northeast section of Morioh, where all the villas are, and I am not married. I work as an employee for the Kame Yu department stores, and I get home every day by 8 PM at the latest. I don't smoke, but I occasionally drink. I'm in bed by 11 PM, and make sure I get eight hours of sleep, no matter what."
prog = re.compile(r'\w{4,}')
prog.findall(string)

['name',
 'Yoshikage',
 'Kira',
 'years',
 'house',
 'northeast',
 'section',
 'Morioh',
 'where',
 'villas',
 'married',
 'work',
 'employee',
 'Kame',
 'department',
 'stores',
 'home',
 'every',
 'latest',
 'smoke',
 'occasionally',
 'drink',
 'make',
 'sure',
 'eight',
 'hours',
 'sleep',
 'matter',
 'what']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
string = "abc.test@gmailcom, @test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
prog = re.compile(r"[\w]@[\w\.-]+(?:\.[\w]+)+")
prog.findall(string)

['t@analyticsvidhya.com', 't@rest.biz']

In [ ]:
string = "abc.test@gmailcom, @test.in, test.first@analyticsvidhya.com, first.test@rest.biz"
massive = string.split(", ")
[re.findall(r"[\w]@[\w\.-]+(?:\.[\w]+)+", massive)[0].split('@')[1] for massive in massive if len(re.findall(r"[\w]@[\w\.-]+(?:\.[\w]+)+", massive)) > 0]

['analyticsvidhya.com', 'rest.biz']